In [56]:
import pandas as pd
from unidecode import unidecode
import numpy as np

In [57]:
# this script creates the database only with production columns, more focused in my work.
crops = ['cocoa', 'coffee', 'corn', 'rice', 'soy', 'sugarcane']
start_cols = ['name', 'state', 'location_type']
data_pam = pd.DataFrame(columns=start_cols)

for crop in crops:

    rename_cols = {
        'Local': 'name',
        'Quantidade produzida (Toneladas)': crop,
        'Área destinada à colheita (Hectares)' : 'area_planted_' + crop,
        'Área plantada (Hectares)': 'area_planted_' + crop,
        'Área colhida (Hectares)': 'area_harvested_' + crop,
        'UF': 'state',
        'Tipo região': 'location_type',
    }

    new_cols = list(set(rename_cols.values()))

    df_temp = pd.read_excel('data/processed/' + crop + '.xlsx').rename(columns=rename_cols)
    df_temp = df_temp[new_cols].drop_duplicates()
    data_pam = data_pam.merge(df_temp, on=['name', 'state', 'location_type'], how='outer')
    
# cleaning strings up

data_pam['name'] = data_pam['name'].apply(unidecode)
data_pam['state'] = data_pam['state'].apply(unidecode)
data_pam['location_type'] = (data_pam['location_type'].apply(unidecode).str
                       .replace('Municipio', 'city')
                       .replace('Microrregiao', 'microregion')
                       .replace('Mesorregiao', 'macroregion')
                       .replace('UF', 'state'))
# putting all productions int64o the same df


data_pam.to_csv('data/processed/data_pam_2022.csv')

In [58]:

rename_cols = {
    'Nome município': 'name',
    'Código IBGE município': 'id_city',
    'Nome microrregião': 'microregion',
    'Código IBGE microrregião': 'id_microregion',
    'Nome mesorregião': 'macroregion',
    'Código IBGE mesorregião': 'id_macroregion',
    'Nome UF': 'state',
    'Código IBGE UF': 'id_state',
    'Sigla UF': 'UF',
    'Nome região': 'region'
}

col_dtypes = {
    'id': 'int64',
    'id_city': 'int64',
    'id_microregion': 'int64',
    'id_macroregion': 'int64',
    'id_state': 'int64',
}

data_ibge_city = pd.read_excel('data/raw/ibge_codes/municipios.xlsx').rename(columns=rename_cols)
data_ibge_city['location_type'] = 'city'
data_ibge_city = data_ibge_city.convert_dtypes(col_dtypes)
data_ibge_city['id'] = data_ibge_city['id_city'] 
str_columns = ['name', 'microregion','macroregion','state','UF','region']
for col in str_columns:
    data_ibge_city[col] = data_ibge_city[col].apply(unidecode)
    
data_ibge_microregion = pd.read_excel('data/raw/ibge_codes/microrregiao.xlsx').rename(columns=rename_cols)
data_ibge_microregion['name'] = data_ibge_microregion['microregion']
data_ibge_microregion['location_type'] = 'microregion'
data_ibge_microregion = data_ibge_microregion.convert_dtypes(col_dtypes)
data_ibge_microregion['id'] = data_ibge_microregion['id_microregion']
str_columns = ['name', 'microregion', 'macroregion','state','UF','region']
for col in str_columns:
    data_ibge_microregion[col] = data_ibge_microregion[col].apply(unidecode)

data_ibge_macroregion = pd.read_excel('data/raw/ibge_codes/mesorregiao.xlsx').rename(columns=rename_cols)
data_ibge_macroregion['name'] = data_ibge_macroregion['macroregion']
data_ibge_macroregion['location_type'] = 'macroregion'
data_ibge_macroregion = data_ibge_macroregion.convert_dtypes(col_dtypes)
data_ibge_macroregion['id'] = data_ibge_macroregion['id_macroregion']
str_columns = ['name','state', 'macroregion', 'UF','region']

for col in str_columns:
    data_ibge_macroregion[col] = data_ibge_macroregion[col].apply(unidecode)

data_ibge_state = pd.read_excel('data/raw/ibge_codes/uf.xlsx').rename(columns=rename_cols)
data_ibge_state['name'] = data_ibge_state['state']
data_ibge_state['location_type'] = 'state'
data_ibge_state = data_ibge_state.convert_dtypes(col_dtypes)
data_ibge_state['id'] = data_ibge_state['id_state']
str_columns = ['name', 'state', 'UF','region']

for col in str_columns:
    data_ibge_state[col] = data_ibge_state[col].apply(unidecode)

data_ibge = pd.concat([data_ibge_city, data_ibge_microregion, data_ibge_macroregion, data_ibge_state])

data_ibge.to_csv('data/processed/data_ibge.csv')



In [59]:
df = data_pam.merge(data_ibge, on=['name', 'location_type', 'state'], how='inner')
crops = ['cocoa', 'coffee', 'corn', 'rice', 'soy', 'sugarcane']
df['total'] = df[crops].sum(axis=1)
df = df.set_index('id')


In [60]:
# finding out urea consumption of each region. At first we'll be considering only the selected crops.
# source - IFA report 2018
kgN_kgurea = 0.46

N_consumption = {
    'rice': 83,
    'corn': 68,
    'soy': 16,
    'sugarcane': 76,
    'coffee': 161,
    'cocoa' : 49
}

df['urea_demand'] = 0
for crop in crops:
    df['urea_demand'] = df['urea_demand'] + N_consumption[crop] * df['area_planted_' + crop] / kgN_kgurea / 1000

# biomass residue to crop yield. Source - de Souza, 2021

residue_data = {
    'rice_husk': {'origin': 'rice', 'ratio': 1.55 * 0.4, 'cost': 60},
    'corn_stover': {'origin': 'corn', 'ratio': 1.68 * 0.4, 'cost': 60},
    'soy_straw' : {'origin': 'soy', 'ratio': 2.3 * 0.3, 'cost': 60},
    'sugarcane_straw': {'origin': 'sugarcane', 'ratio': 0.22 * 0.59, 'cost': 60},
    'sugarcane_bagasse': {'origin': 'sugarcane', 'ratio': 0.22 * 0.25, 'cost': 40},
    'coffee_husk':  {'origin': 'coffee', 'ratio': 0.59 * 0.50, 'cost': 60},
    'cocoa_residue': {'origin': 'cocoa', 'ratio': 0.59 * 0.50, 'cost': 60}, # sem dados
}

for residue in residue_data:
    df[residue] = df[residue_data[residue]['origin']] * residue_data[residue]['ratio']
    df[residue+'_cost'] = residue_data[residue]['cost']

df['urea_price'] = 310

In [61]:
# for each microregion, finding out what is the city with maximum production. that will be the reference city for distance purposes.


df.loc[df['location_type'] == 'city', 'location_id'] = df.loc[df['location_type'] == 'city'].index

microregions = df.loc[df['location_type'] == 'microregion'].index
for microregion in microregions:
    df.loc[microregion, 'location_id'] = df['total'].loc[(df['location_type'] == 'city') & (df['id_microregion'] == microregion)].idxmax()


macroregions = df.loc[df['location_type'] == 'macroregion'].index
for macroregion in macroregions:
    df.loc[macroregion, 'location_id'] = df['total'].loc[(df['location_type'] == 'city') & (df['id_macroregion'] == macroregion)].idxmax()

states = df.loc[df['location_type'] == 'state'].index
for state in states:
    df.loc[state, 'location_id'] = df['total'].loc[(df['location_type'] == 'city') & (df['id_state'] == state)].idxmax()




In [62]:
distance_matrix = pd.read_csv('data/raw/distance_matrix/matriz_distancias.csv', index_col=['origem', 'destino']).drop(columns='tempo')['distancia'].unstack()
distance_matrix.index.name = 'origin'

correct_keys = list(df.index)
wrong_keys = (np.array(list(df.index)) / 10).astype(int)
rename_dict = {i: j for i, j in zip (wrong_keys, correct_keys)}
distance_matrix = distance_matrix.rename(index=rename_dict, columns=rename_dict)
distance_matrix = distance_matrix

In [63]:
# exporting final version of both the main database and the distance matrix
df.to_pickle('data/processed/location_db.p')
distance_matrix.to_pickle('data/processed/distance_matrix.p')